## Inspired from 1st team ([github](https://github.com/alipay/cvpr2020-plant-pathology)) in [Plant Pathology 2020 - FGVC7](http://www.kaggle.com/c/plant-pathology-2020-fgvc7), it is interesting to display the CAM to track model performance.

### Thank my teammate @toxu for sharing me with this repo.

# What is Class Activation Map (CAM) ? 

### A Class Activation map for a particular category indicates the particular region used by CNN to identify the output class.

### The CNN model is composed of numerous convolutionary layers and we perform global average pooling just before the final output layer. To get the desired output, the resulting features are fed to a fully connected layer with softmax activation. By projecting the output layer weights back into the convolutionary maps derived from the last Convolution Layer the importance of the image regions is identifiable.

ref: 

https://medium.com/intelligentmachines/implementation-of-class-activation-map-cam-with-pytorch-c32f7e414923

https://arxiv.org/abs/1512.04150

# Using general structure at this competition for easily understanding the pipeline

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import shutil
import albumentations
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
from scipy.special import softmax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## replace it with your model path
MODEL_DIR = '../input/cassava-resnext/' 

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test['filepath'] = test.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/test_images', f'{x}'))

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train['filepath'] = train.image_id.apply(lambda x: os.path.join('../input/cassava-leaf-disease-classification/train_images', f'{x}'))

In [ ]:
class CFG:
    num_workers=4
    model_name='seresnext50_32x4d'
    size=512
    batch_size=1
    target_size=5
    target_col='label'
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    seed=123

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TRAIN_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_transforms(*, data):
    if data == 'no-tta':
        return A.Compose([
            A.Resize(CFG.size, CFG.size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    
    if data == 'tta':
        return A.Compose([
            RandomResizedCrop(CFG.size, CFG.size),
#             A.Resize(CFG.size, CFG.size),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

In [ ]:
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if 'legacy' in model_name:
            n_features = self.model.last_linear.in_features
            self.model.last_linear = nn.Linear(n_features,CFG.target_size)
        elif 'res' in model_name:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, CFG.target_size)
        elif 'eff' in model_name:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, CFG.target_size)
            
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(CFG.seed)

In [ ]:
test_dataset = TestDataset(test, transform=get_transforms(data='no-tta'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)

train_dataset = TestDataset(train, transform=get_transforms(data='no-tta'))
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)

In [ ]:
model = CustomResNext(CFG.model_name, pretrained=False)
model.to(device)

## load one of your model
state = torch.load(MODEL_DIR+f'{CFG.model_name}_fold_{0}')
model.load_state_dict(state)

In [ ]:
## remove gap and final linear layer
## if you use eff or other models, this line would be different
## try to change by yourself, and you will learn and understand
mod = nn.Sequential(*list(model.model.children())[:-2])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__() 
        self.fc=nn.Linear(2048,5)

    
    def forward(self,x):     
        x=x.view(2048,16*16).mean(1).view(1,-1) ## for image size 512 x 512, the feature map size is 16 x 16
        x=self.fc(x)
        return  F.softmax(x,dim=1)

In [ ]:
## concat feature map layer and linear layer
model=nn.Sequential(mod,Net())

In [ ]:
params = list(Net().parameters())
weight = np.squeeze(params[-2].data.numpy())

In [ ]:
params[-2].shape

In [ ]:
def return_CAM(feature_conv, weight, class_idx):
    size_upsample = (CFG.size,CFG.size)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        beforeDot =  feature_conv.reshape((nc, h*w))
        cam = np.matmul(weight[idx], beforeDot)
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [ ]:
## for example, get some of images to show
imgs = []
for i, (images) in enumerate(train_loader):
    if i >= 10:
        break
    imgs.append(images)

In [ ]:
i = 7
img = imgs[i]
model.to(device)
img = img.to(device)
logit = model(img)

h_x = F.softmax(logit, dim=1).data.squeeze()
 
probs, idx = h_x.sort(0, True)
probs = probs.cpu().detach().numpy()
idx = idx.cpu().numpy()


features_blobs = mod(img)
features_blobs1 = features_blobs.cpu().detach().numpy()
CAMs = return_CAM(features_blobs1, weight, [idx[0]])


img = cv2.imread(train.loc[i, 'filepath'])
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)

result = heatmap * 0.3 + img * 0.7
# cv2.imwrite("sample.jpg", result)

plt.figure(figsize=(16,8))
plt.imshow(result.astype('uint8'))